# Lifetimes & Annotations

## Segments of memory in Rust

* **Stack** - used to hold state related to currently executing functions:
  - The parameters passed to function
  - Local variables defined within function
  - Temporary values calculated within function
  - The return address of function

In [ ]:
fn add(x: i32, y: i32) -> i32 {
    x + y
}

fn caller() -> i32 {
    let a = 10;
    let b = 20;
    let result = add(a, b);
    result
}

fn main() {
    let sum = caller();
    println!("The sum is: {}", sum);
}

main();

The sum is: 30



* **Heap** - for dynamically allocated memory controlled via smart pointers or data structures (`Vec`, `String`, `Box`, etc)


In [3]:
fn main() {
    let text = String::from("Hello, Rust!");
    let number_on_heap = Box::new(42);

    println!("Text: {}, Number on heap: {}", text, number_on_heap);
}

main();

Text: Hello, Rust!, Number on heap: 42


* **Static** - static variables with fixed location in memory for the duration of the program

In [4]:
static GREETINGS: &str = "Hello, World!";

fn main() {
    println!("{}", GREETINGS);
}

main();

Hello, World!


## Scopes and Lifetimes

The lifetime of a variable is the region of code where that variable is valid. In Rust, lifetimes are often associated with references to ensure memory safety.

The **lifetime** of an item on stack is the period where item is guaranteed to stay in the same place in memory. This is exactly the period where **reference** (pointer) to that item **is valid**.
The lifetime starts when item is created, and extends to where is:

* **dropped** (goes out of scope) or
* **moved** (ownership transferred to another variable)

```rust

In [12]:
#[derive(Debug)]
struct Value {
    content: i32
}

fn consume_value(val: Value) {
    println!("Consuming value: {:?}", val);
}

fn main() {
    let val1 = Value { content: 100 }; // val1 is created here
    let val2 = Value { content: 200 }; // val2 is created here
    println!("Value 1: {:?}, Value 2: {:?}", val1, val2);

    consume_value(val2); // val2 is moved here
} // val1 is dropped here

main();

Value 1: Value { content: 100 }, Value 2: Value { content: 200 }
Consuming value: Value { content: 200 }


Borrowchecker tracks lifetimes of all references in the program to ensure that references do not outlive the data they point to.

### Example: Dangling Reference 1

In [6]:
fn main() {
    let v: &Value;
    {
        let val = Value { content: 300 };
        v = &val; // Error: val does not live long enough
    }
    println!("Value: {:?}", v);
}

main();

Error: `val` does not live long enough

### Example: Dangling Reference 2

In [7]:
fn return_ref_(v: &Value) -> &i32 {
   return &v.content;
}

fn main() {
    {   // valid usage
        let val = Value { content: 400 };
        let ref_content: &i32 = return_ref_(&val);
        println!("Reference to content: {}", ref_content);
    }

    let ref_content: &i32 = return_ref_(&Value{content: 500}); // invalid usage - temporary value does not live long enough - borrowchecker rejects the code
    println!("Reference to content: {}", ref_content);
}

main();

Error: temporary value dropped while borrowed

### Non-lexical Lifetimes (NLL)

* NLL is a feature that allows the Rust borrow checker to be more flexible in how it determines the lifetimes of references.
* If the compiler can prove to itself that there is no use of a reference beyond a certain point in the code, then it treats the endpoint of the reference’s lifetime as the last place it’s used, rather than at the end of the enclosing scope

In [8]:
fn main() {
    let mut text = String::from("hello world");

    // create a mutable reference to a portion of the string
    let greeting = &mut text[..5];
    greeting.make_ascii_uppercase();
    // no use of grreeting after this point

    let ref_to_text = &text; // now this is allowed
    println!("Text: {}", ref_to_text);
}

main();

Text: HELLO world


## Returning references

* You can’t return a reference to local variable that only lives inside a function - it would be a *dangling reference*

In [9]:
fn create_dangling_reference() -> &'static str {
    let s = String::from("Hello, world!");
    &s
}

Error: cannot return reference to local variable `s`

* You can return a reference to static variable (like string literals) - return type needs `'static` *lifetime annotation*

In [10]:
fn get_greetings() -> &'static str {
    let s = "Hello, world!";
    s
}

## Lifetime annotations

* Most of the time, Rust takes care of lifetimes for you, but sometimes, it needs a bit of extra help. This extra help is called a **lifetime annotation**, which means “extra lifetime information.”
* Liftetime annotations don't change the lifetimes of any references, they just describe the relationships of the lifetimes of multiple references.
* Lifetime annotations syntax: the names of lifetime parameters must start with an apostrophe (') and are usually all lowercase and very short, like this: `'a`, `'b`, and so on. But the names of lifetime parameters are arbitrary and have no meaning.

### Inferring lifetimes

* Rust has a **lifetime elision** rules, which are a set of three rules that the compiler follows to figure out what lifetimes references have when there aren't explicit annotations.
* The first rule applies to input lifetimes, and the second and third rules apply to output lifetimes. If the compiler gets to the end of the three rules and there are still references for which it can’t figure out lifetimes, the compiler will stop with an error.

* **The first rule**: each parameter that is a reference gets its own lifetime parameter:
  * a function with one parameter gets one lifetime parameter: `fn foo<'a>(x: &'a i32);` 
  * a function with two parameters gets two separate lifetime parameters:`fn foo<'a, 'b>(x: &'a i32, y: &'b i32);` and so on.
* **The second rule**: if there is exactly one input lifetime parameter, that lifetime is assigned to all output lifetime parameters: 
  * `fn foo<'a>(x: &'a i32) -> &'a i32;`
* **The third rule**: if there are multiple input lifetime parameters, but one of them is `&self` or `&mut self` because this is a method, the lifetime of `self` is assigned to all output lifetime parameters: 
  * `fn foo<'a, 'b>(&'a self, x: &'b i32) -> &'a i32;`

In [11]:
fn print_value(v: &Value) {
    println!("Value.content: {}", v.content);
}

can be explicitly written as:

In [12]:
fn print_value<'a>(v: &'a Value) {
    println!("Value.content: {}", v.content);
}

let val = Value { content: 600 };
print_value(&val);

Value.content: 600


### Static Lifetime

What happens when we have no input lifetimes, but we want to return a reference? In this case, we can use the `'static` lifetime, which means that the reference can live for the entire duration of the program.

In [20]:
static THE_ANSWER: Value = Value { content: 42 };

fn the_answer() -> &'static Value {
    &THE_ANSWER
}

fn main() {
    let answer = the_answer();
    println!("The answer is: {}", answer.content);
}

main();

The answer is: 42


* The Rust compiler guarantees that a `static` item always has the same address for the entire duration of the program and never moves. This means that a reference to a static item has a ``'static`` lifetime.
* Other objects with `'static` lifetime:
  * String literals
  * Constants - but not if they implement `Drop` trait
  * Leaked objects - using `Box::leak` or similar functions



In [23]:
{
    let on_heap = Box::new(Value{content: 665});
    let static_ref: &'static Value = Box::leak(on_heap);
    print_value(static_ref);
}

Value.content: 665


()

### Returning references

When a function takes a single reference as an argument, and returns a single reference, Rust assumes that the two must have the same lifetime (see *the second rule*).

In [24]:
fn ref_to_min(data: &mut [i32]) -> &mut i32 {
    // let min = data.iter_mut().min().unwrap();
    
    let mut index: usize = 0;
    for (i, value) in data.iter().enumerate() {
        if *value < data[index] {
            index = i;
        }
    } 

    &mut data[index]
}

In [25]:
fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);
    *the_smallest = 0;

    println!("{:?}", data);
}

main();

[42, 665, 0, 44, 99]


In [26]:
let s;

{
    let mut data = [9, 4, 1, 0, 1, 4, 9];
    s = ref_to_min(&mut data);
}

println!("{}", *s);

Error: The variable `s` contains a reference with a non-static lifetime so
can't be persisted. You can prevent this error by making sure that the
variable goes out of scope - i.e. wrapping the code in {}.

## Structs Containing References (Annotations)

In [27]:
struct View {
    value: &i32
}

fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);

    let view = View { value: the_smallest };
    println!("{}", view.value);
}

Error: missing lifetime specifier

Whenever a reference type appears inside another type’s definition, you must write out its lifetime.

In [28]:
struct View<'a> {
    value: &'a i32
}

fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);
    let view = View { value: the_smallest };
    println!("{}", view.value);
}

main();

1


Now the `View` type has a lifetime, just as reference types do. Each value you create of type `Value` gets a fresh lifetime `'a`, which becomes constrained by how you use the value. The lifetime of any reference you store in `ref_value` had better enclose `'a`, and `'a` must outlast the lifetime of wherever you store the `View`.

In [29]:
struct View<'a> {
    value: &'a i32
}

#[allow(unused_variables)]
fn main() {
    let mut bag: Vec<View> = Vec::new();
    {
        let mut data = [42, 665, 1, 44, 99];
        let the_smallest: &mut i32 = ref_to_min(&mut data);

        let view = View { ref_value: the_smallest };
        println!("{}", view.ref_value);
        bag.push(view);
    }

    println!("{}", bag[0].ref_value);
}

main();

Error: struct `View<'_>` has no field named `ref_value`

Error: no field `ref_value` on type `View<'_>`

Error: no field `ref_value` on type `View<'_>`

When we want to place `View` inside of another `struct`, we must specify the lifetime of the reference it contains:

In [30]:
struct Record {
    v: View<'static>
}

// or

struct MinView<'a> {
    v: View<'a>
}

A type’s lifetime parameters always reveal whether it contains references with interesting (that is, non-`'static`) lifetimes, and what those lifetimes can be

In [31]:
fn get_min<'a>(data: &'a mut [i32]) -> MinView<'a> {
    let min: &'a mut i32 = data.iter_mut().min().unwrap();
    MinView { v: View { value: min } }
}

Without looking into the definition of the `MinView` type at all, we can tell that, if we receive a `MinView` from `get_min`, whatever references it contains must point into the input slice we passed in, and nowhere else (except perhaps at 'static values).

## Distinct Lifetimes


In [2]:
#[allow(unused_variables)]
struct Record<'a> {
    x: &'a i32,
    y: &'a i32
}

In [3]:
#[allow(unused_variables)]
fn main() {
    let x = 42;
    let r: &i32;
    {
        let y = 665;
        let local = Record { x: &x, y: &y };
        r = local.x;
    }
    println!("{}", r);
}

main();

Error: `y` does not live long enough

This code doesn’t create any dangling pointers. The reference to `y` stays in `local`, which goes out of scope before `y` does. The reference to `x` ends up in `r`, which doesn’t outlive `x`.

If you try to compile this, however, Rust will complain that `y` does not live long enough, even though it clearly does. Why is Rust worried? If you work through the code carefully, you can follow its reasoning:

* Both fields of `Record` are references with the same lifetime `'a`, so Rust must find a single lifetime that works for both `local.x` and `local.y`.
* We assign `r = local.x`, requiring `'a` to enclose `r’s` lifetime.
* We initialized `local.y` with `&y`, requiring `'a` to be no longer than `y`’s lifetime.

These constraints are impossible to satisfy: no lifetime is shorter than `y`’s scope, but longer than `r`’s.

The problem arises because both references in `Record` have the same lifetime `'a`. Changing the definition of `Record` to let each reference have a distinct lifetime fixes everything:

In [34]:
struct Record<'a, 'b> {
    x: &'a i32,
    y: &'b i32,
}

impl<'a, 'b> Record<'a, 'b> {
    fn print(&self) {
        println!("x: {}, y: {}", self.x, self.y);
    }
}

In [35]:
#[allow(unused_variables)]
fn main() {
    let x = 42;
    let r;
    {
        let y = 665;
        let local = Record { x: &x, y: &y };
        r = local.x;
        local.print();
    }
    println!("{}", r);
}

main();

x: 42, y: 665
42


With this definition, `local.x` and `local.y` have independent lifetimes. What we do with `local.x` has no effect on what we store in `local.y`, so it’s easy to satisfy the constraints now: `'a` can simply be `r`’s lifetime, and `'b` can be `local`’s. (`y`’s lifetime would work too for `'b`, but Rust tries to choose the smallest lifetime that works). 

Function signatures can have similar effects. Suppose we have a function like this:

In [4]:
fn foobar_tight<'a>(x: &'a i32, y: &'a i32) -> &'a i32 { // perhaps too restrictive
    x
}

In [5]:
fn foobar<'a, 'b>(x: &'a i32, y: &'b i32) -> &'a i32 { // looser
    x
}

Try the simplest possible definition first, and then loosen restrictions until the code compiles. Since Rust won’t permit the code to run unless it’s safe, simply waiting to be told when there’s a problem is a perfectly acceptable tactic.

### Anonymous Lifetimes

The anonymous lifetime '_ allows you to mark an elided lifetime as being present, without having to fully restore all of the lifetime names:

In [9]:
struct MinElement<'a> {
    index: usize,
    value: &'a i32
}

fn find_min(data: &[i32]) -> MinElement<'_> {
    let mut index: usize = 0;
    for (i, value) in data.iter().enumerate() {
        if *value < data[index] {
            index = i;
        }
    } 

    MinElement { index, value: &data[index] }
}

fn main() {
    let data = [42, 665, 1, 44, 99];
    let min_view = find_min(&data);
    println!("Minimum value is: {} at index {}", min_view.value, min_view.index);
}

main();

Minimum value is: 1 at index 2


# Borrow Rules

* There are three ways to have access to data in Rust:
  
  * No references (**ownership**) - CRUD - Create, Read, Update, Drop
    * only the item's owner can **move** the item
  
  * A **mutable reference** can read from underlying data and also modify it 
  * A reference that can only read from underlying data (**shared reference**)

## Moving Ownership

* You cannot move out an item that is behind a mutable reference!!!

In [7]:
#[derive(Debug)]
struct Item {
    content: i32
}

In [8]:

fn replace(item: &mut Option<Item>, new_item: Item) -> Option<Item> {
    let previous_value = *item;
    *item = Some(new_item);
    previous_value
}

Error: cannot move out of `*item` which is behind a mutable reference

* We can, however, replace the item behind a mutable reference with a new item, and take ownership of the previous item. This is done using the `std::mem::replace` function:

In [12]:
fn replace(item: &mut Option<Item>, new_item: Item) -> Option<Item> {
    std::mem::replace(item, Some(new_item))
}

fn main() {
    let mut item_option: Option<Item> = Some(Item { content: 10 });

    let old_item = replace(&mut item_option, Item { content: 20 });
    println!("Old item: {:?}, New item: {:?}", old_item, item_option);
}

main();

Old item: Some(Item { content: 10 }), New item: Some(Item { content: 20 })


# Borrow Rules

There are two key rules to remember about borrowing references in Rust:

1. References must always be valid. This means they cannot outlive the data they point to.
   * scope of reference ≤ lifetime of data it points to
2. You can have either:
   * one mutable reference to the item or
   * any number of immutable references to the item at a time.

The borrowing rules allow the compiler to make better decisions around aliasing: tracking when two different pointers may or may not refer to the same underlying item in memory. This enables optimizations that would otherwise be unsafe if aliasing were possible.

In [25]:
fn extend(vec: &mut Vec<i32>, slice: &[i32]) {
    for item in slice {
        vec.push(*item);
    }
}

let mut data = vec![1, 2, 3, 4];
let exta_data_1 = vec![5, 6, 7];
let exta_data_2 = vec![8, 9, 10];

extend(&mut data, &exta_data_1);
extend(&mut data, &exta_data_2);

println!("{:?}", data);

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Let's try extend a vector with a slice of its own elements:

In [26]:
extend(data, &data);

Error: mismatched types

Error: unused variable: `y`

Error: unused variable: `y`

### Rust Rules for Mutation & Sharing

* **Shared access** is **read-only access**. Values borrowed by shared references are read-only. Across the lifetime of a shared reference, neither its referent, nor anything reachable from that referent, can be changed by anything. There exist no live mutable references to anything in that structure; its owner is held read-only; and so on. It’s immutable

* **Mutable access** is **exclusive access**. A value borrowed by a mutable reference is reachable exclusively via that reference. Across the lifetime of a mutable reference, there is no other usable path to its referent, or to any value reachable from there. The only references whose lifetimes may overlap with a mutable reference are those you borrow from the mutable reference itself.

In [2]:
{
    let mut x = 10;
    let rx1 = &x;
    let rx2 = &x;     // OK: multiple shared borrows permitted

    x += 10;         // Error: cannot assign to `x` because it is borrowed

    println!("{x}, {rx1}, {rx2}");
}

Error: cannot assign to `x` because it is borrowed

In [3]:
{
    let mut x = 10;
    let rx = &x;
    
    let mx = &mut x; // Error: cannot borrow `x` as mutable because it is also borrowed as immutable

    println!("{x}, {rx}, {mx}");
}

Error: cannot borrow `x` as mutable because it is also borrowed as immutable

Error: cannot borrow `x` as immutable because it is also borrowed as mutable

In [ ]:
{
    let mut y = 20;
    let my1 = &mut y;
    let my2 = &mut y;  // Error: cannot borrow as mutable more than once

    println!("{y}, {my1}, {my2}");
}

Error: cannot borrow `y` as mutable more than once at a time

Error: cannot borrow `y` as immutable because it is also borrowed as mutable

It is **OK to reborrow a shared reference from a shared reference**:

In [ ]:
{
    let record = (42, 665);
    let r = &record;
    let r0 = &r.0;  // OK: reborrowing shared as shared
}

()

In [7]:
{
    let record = (42, 665);
    let r = &record;
    let m1 = &mut r.1;  // Error: cannot reborrow as mutable because it is also borrowed as immutable
}

Error: unused variable: `m1`

Error: cannot borrow `r.1` as mutable, as it is behind a `&` reference

You can reborrow from a mutable reference:

In [12]:
{
    let mut record = (42, 665);
    let m = &mut record;
    let m0 = &mut m.0; 
    *m0 = 123; // OK: mutable borrow is unique

    let r1 = &m.1; // Ok: reborrowing mutable as shared
    println!("{m0}, {r1}");
}

123, 665


()